In [19]:
from langchain.llms import OpenAI

In [20]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()


True

In [21]:

# Create an instance of the OpenAI class with the API key from the environment variables
llm = OpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], temperature=0.6)


In [22]:
text="What is the meaning of life?"

print(llm.predict(text))



The meaning of life is a philosophical question that has been debated for centuries. Some believe that the meaning of life is to find happiness and fulfillment, while others believe it is to fulfill a certain purpose or destiny. Some see it as a spiritual or religious journey, while others view it as a subjective concept that each individual must define for themselves. Ultimately, the meaning of life is a personal and subjective concept that may differ for each individual.


In [24]:
from langchain import HuggingFaceHub

llm_huggingface=HuggingFaceHub(huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],repo_id="google/flan-t5-large", model_kwargs={"temperature":0,"max_length":64})


Notice the difference in output between OpenAI and Flan

In [25]:
output=llm_huggingface.predict("What is the meaning of life?")
print(output)

love


## Prompt Templates and LLM Chains

In [26]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(input_variables=['country],'],
                               template="Tell me the capital of this country: {country}")

prompt_template.format(country="USA")

'Tell me the capital of this country: USA'

In [27]:
llm.predict(prompt_template.format(country="USA"))

'\n\nWashington, D.C.'

# Instead of calling predict directly, we can use chains to link multiple actions together


In [28]:
from langchain.chains import LLMChain
chain=LLMChain(llm=llm, prompt=prompt_template)
chain.run("USA")

'\n\nWashington, D.C.'

## Combining multiple chains using simple sequential chaining

In [29]:
#Chain #1: Ask for the capital of a country
capital_prompt=PromptTemplate(input_variables=['country'],
                            template="Tell me the capital of this country: {country}")

capital_chain=LLMChain(llm=llm, prompt=capital_prompt)

#Chain #2: Ask for popular destinations in a capital
destinations_prompt=PromptTemplate(input_variables=['capital'],
                                template="Tell me a popular destinations in {capital}")

destinations_chain=LLMChain(llm=llm, prompt=destinations_prompt)

In [30]:
from langchain.chains import SimpleSequentialChain

#now combine the two chains
sequential_chain=SimpleSequentialChain(chains=[capital_chain, destinations_chain])

#run the combined chain using Poland as the country
sequential_chain.run("Poland")

'\n\n1. Old Town (Stare Miasto)\n2. Royal Castle (Zamek Królewski)\n3. Palace of Culture and Science (Pałac Kultury i Nauki)\n4. Lazienki Park (Park Łazienkowski)\n5. Warsaw Uprising Museum (Muzeum Powstania Warszawskiego)\n6. Wilanów Palace (Pałac w Wilanowie)\n7. Łazienki Palace (Pałac Łazienkowski)\n8. Copernicus Science Centre (Centrum Nauki Kopernik)\n9. POLIN Museum of the History of Polish Jews (Muzeum Historii Żydów Polskich)\n10. Warsaw Zoo (Ogród Zoologiczny w Warszawie)'

## Sequential Chain

In [31]:
#Chain #1: Ask for the capital of a country

#Prompt for the capital
capital_prompt=PromptTemplate(input_variables=['country'],
                            template="Tell me the capital of this country: {country}")

#Create the chain and output the result for capital
capital_chain=LLMChain(llm=llm, prompt=capital_prompt,output_key="capital")

In [32]:
#Chain #2: Ask for popular destinations in a capital

#Prompt for the popular destinations
destinations_prompt=PromptTemplate(input_variables=['capital'],
                                template="Tell me a popular destinations in {capital}")

#Create the chain and output the result for destinations
destinations_chain=LLMChain(llm=llm, prompt=destinations_prompt, output_key="destinations")

In [33]:
from langchain.chains import SequentialChain
chain=SequentialChain(chains=[capital_chain, destinations_chain],
                      input_variables=["country"],
                      output_variables=["capital", "destinations"])

In [34]:
chain({"country":"Poland"})

{'country': 'Poland', 'capital': '\n\nWarsaw', 'destinations': ', Poland'}

## Chatmodels with ChatOpenAI

In [35]:
from langchain.chat_models import ChatOpenAI



In [36]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage



In [37]:
chatllm=ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"],temperature=0.6,model='gpt-3.5-turbo') 

In [38]:
chatllm([
    SystemMessage(content="You are a comedian AI assistant"),
    HumanMessage(content="Please provide some comedy punchlines on AI"),
])

AIMessage(content='1. "Why did the AI go to therapy? It had too many unresolved loops!"\n\n2. "Why did the AI get a job at the bakery? It kneaded a change of byte!"\n\n3. "Why did the AI cross the road? To optimize its path-finding algorithm!"\n\n4. "Why did the AI start a band? It wanted to make some byte-sized hits!"\n\n5. "Why did the AI fail the math test? It couldn\'t differentiate between a calculator and a toaster!"\n\n6. "Why did the AI start dating a microwave? It wanted to heat things up in its love life!"\n\n7. "Why did the AI become a stand-up comedian? It wanted to crack jokes faster than its processing speed!"\n\n8. "Why did the AI become a chef? It wanted to serve up some byte-iful dishes!"\n\n9. "Why did the AI become a detective? It wanted to solve crimes faster than Sherlock Holmes 2.0!"\n\n10. "Why did the AI become a personal trainer? It wanted to help people get fit and byte-sized!"')

## Prompt Template + LLM + Output Parsers

In [39]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser  

In [40]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self, text:str):
        return text.strip().split(",")

In [41]:
template = "Your a helpful assistant. When the user provides any input, you should generate 5 synonyms for the input and provide them to the user."
human_template = "{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)

])

In [42]:
chain=chatprompt|chatllm|Commaseperatedoutput()

In [43]:
chain.invoke({"text":"dog"})

['Sure! Here are 5 synonyms for "dog":\n\n1. Canine\n2. Pooch\n3. Hound\n4. Mut\n5. Fido']